# 03 — Slot Filling & Query Builder
**Date:** 2025-08-09

Turn extracted entities into a `QuerySpec`, then into LINQ filters.


In [ ]:
#r "nuget: Microsoft.Recognizers.Text, 1.8.13"
#r "nuget: Microsoft.Recognizers.Text.DateTime, 1.8.13"
#r "nuget: FuzzySharp, 2.0.2"
#r "nuget: RestSharp, 112.1.0"
#r "nuget: Microsoft.Extensions.Caching.Memory, 9.0.8"
#r "nuget: System.Text.Json, 9.0.0"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text.Json;
using Microsoft.Recognizers.Text;
using Microsoft.Recognizers.Text.DateTime;
using FuzzySharp;
using RestSharp;

public enum Intent { GetContactInfo, FilterByHireDate, FilterByRole, Unknown }

public record Slots(
    string[]? Names = null,
    DateTime? Date = null,
    (DateTime Start, DateTime End)? Range = null,
    string? Operator = null,
    string? Department = null,
    string? Role = null
);

public record QuerySpec(Intent Intent, Slots Slots);

public record Employee(
    string DisplayName,
    string Email,
    string Department,
    string Role,
    DateTime OriginalHireDate
);

var employees = new List<Employee> {
    new("Rick Sanchez",   "rick.sanchez@company.com",   "Engineering", "Staff Engineer",  new DateTime(2015,  5, 10)),
    new("Morty Smith",    "morty.smith@company.com",    "Engineering", "Engineer I",      new DateTime(2023, 10, 12)),
    new("Summer Smith",   "summer.smith@company.com",   "Product",     "PM",              new DateTime(2021,  2,  1)),
    new("Beth Smith",     "beth.smith@company.com",     "HR",          "HR Manager",      new DateTime(2019,  7,  3)),
    new("Jerry Smith",    "jerry.smith@company.com",    "Sales",       "Account Manager", new DateTime(2022,  9, 15))
};

Console.WriteLine($"Loaded demo employees: {employees.Count}");


In [ ]:
static Intent ClassifyIntent(string query)
{
    var q = query.ToLowerInvariant();
    if (q.Contains("email") || q.Contains("contact")) return Intent.GetContactInfo;
    if (q.Contains("hire") && (q.Contains("before") || q.Contains("after") || q.Contains("between"))) return Intent.FilterByHireDate;
    if (q.Contains("manager") || q.Contains("engineer") || q.Contains("director") || q.Contains("role")) return Intent.FilterByRole;
    return Intent.Unknown;
}
public record DateExtraction(DateTime? Date, (DateTime Start, DateTime End)? Range, string? Operator);
static DateExtraction ExtractDates(string query)
{
    var results = DateTimeRecognizer.RecognizeDateTime(query, Culture.English);
    var values = new List<DateTime>();
    foreach (var r in results)
    {
        if (!r.Resolution.TryGetValue("values", out var valsObj)) continue;
        var vals = valsObj as List<Dictionary<string, string>>;
        if (vals == null) continue;
        foreach (var v in vals)
        {
            if (v.TryGetValue("value", out var s) && DateTime.TryParse(s, out var dt))
                values.Add(dt);
            else if (v.TryGetValue("start", out var s1) && v.TryGetValue("end", out var s2)
                     && DateTime.TryParse(s1, out var d1) && DateTime.TryParse(s2, out var d2))
                return new DateExtraction(null, (d1, d2), "between");
        }
    }
    string? op = null;
    var lower = query.ToLowerInvariant();
    if (lower.Contains("before")) op = "before";
    else if (lower.Contains("after")) op = "after";
    else if (lower.Contains("between")) op = "between";
    if (values.Count >= 2) return new DateExtraction(null, (values.Min(), values.Max()), "between");
    if (values.Count == 1) return new DateExtraction(values[0], null, op);
    return new DateExtraction(null, null, null);
}
var lexiconJson = @"{
  ""departments"": { ""engineering"": [""engineering"",""eng"",""platform"",""product engineering""], ""hr"": [""hr"",""human resources""], ""sales"": [""sales"",""bizdev"",""business development""], ""product"": [""product"",""pm"",""program management""] },
  ""roles"": { ""manager"": [""manager"",""supervisor"",""team lead"",""lead""], ""engineer"": [""engineer"",""developer"",""dev"",""software engineer""], ""pm"": [""pm"",""product manager"",""program manager""] }
}";
using System.Text.Json.Nodes;
var lexicon = JsonNode.Parse(lexiconJson)!.AsObject();
static string? MapAlias(System.Text.Json.Nodes.JsonObject section, string input)
{
    var q = input.ToLowerInvariant();
    foreach (var kvp in section)
    {
        var canon = kvp.Key;
        var aliases = kvp.Value!.AsArray().Select(n => n!.ToString().ToLowerInvariant());
        if (aliases.Contains(q) || canon.ToLowerInvariant() == q) return canon;
        var best = FuzzySharp.Process.ExtractOne(q, aliases.ToList());
        if (best != null && best.Score >= 90) return canon;
    }
    return null;
}
string? MapDepartment(string text) => MapAlias((System.Text.Json.Nodes.JsonObject)lexicon["departments"]!, text);
string? MapRole(string text) => MapAlias((System.Text.Json.Nodes.JsonObject)lexicon["roles"]!, text);
static List<string> ExtractCandidateNames(string query)
{
    var seps = new [] {","," and "," & "};
    var temp = query.ToLowerInvariant();
    foreach (var s in seps) temp = temp.Replace(s, "|");
    return temp.Split("|", StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries).ToList();
}
static List<Employee> MatchNames(string query, IEnumerable<Employee> all, int minScore = 85, int topK = 3)
{
    var tokens = ExtractCandidateNames(query);
    var results = new List<Employee>();
    foreach (var t in tokens)
    {
        var best = FuzzySharp.Process.ExtractTop(t, all.ToList(), e => e.DisplayName, limit: topK);
        var accepted = best.FirstOrDefault(b => b.Score >= minScore);
        if (accepted != null && !results.Any(r => r.DisplayName == accepted.Value.DisplayName))
            results.Add(accepted.Value);
    }
    return results;
}


In [ ]:
public static QuerySpec BuildQuerySpec(string query)
{
    var intent = ClassifyIntent(query);
    var names = MatchNames(query, employees).Select(e => e.DisplayName).ToArray();
    var dates = ExtractDates(query);

    string? dept = null; string? role = null;
    var words = query.ToLowerInvariant().Split(' ', StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries);
    foreach (var w in words)
    {
        dept ??= MapDepartment(w);
        role ??= MapRole(w);
    }

    var slots = new Slots(
        Names: names.Length > 0 ? names : null,
        Date: dates.Date,
        Range: dates.Range,
        Operator: dates.Operator,
        Department: dept,
        Role: role
    );

    return new QuerySpec(intent, slots);
}

var spec = BuildQuerySpec("show managers hired before 2021 in engineering, also email rick and summer");
Console.WriteLine(spec);


In [ ]:
public static IEnumerable<Employee> ExecuteQuery(QuerySpec spec, IEnumerable<Employee> all)
{
    IEnumerable<Employee> q = all;

    if (spec.Slots.Names is { Length: > 0 })
        q = q.Where(e => spec.Slots.Names!.Contains(e.DisplayName));

    if (!string.IsNullOrWhiteSpace(spec.Slots.Department))
        q = q.Where(e => e.Department.Equals(spec.Slots.Department, StringComparison.OrdinalIgnoreCase));

    if (!string.IsNullOrWhiteSpace(spec.Slots.Role))
        q = q.Where(e => e.Role.Contains(spec.Slots.Role!, StringComparison.OrdinalIgnoreCase));

    if (spec.Slots.Operator == "before" && spec.Slots.Date is DateTime d1)
        q = q.Where(e => e.OriginalHireDate < d1);
    else if (spec.Slots.Operator == "after" && spec.Slots.Date is DateTime d2)
        q = q.Where(e => e.OriginalHireDate > d2);
    else if (spec.Slots.Operator == "between" && spec.Slots.Range is (DateTime s, DateTime e2))
        q = q.Where(e => e.OriginalHireDate >= s && e.OriginalHireDate <= e2);

    return q.ToList();
}

var spec1 = BuildQuerySpec("employees hired before 2024 in engineering");
var res1 = ExecuteQuery(spec1, employees);
foreach (var e in res1) Console.WriteLine($"{e.DisplayName} — {e.Email} — {e.OriginalHireDate:yyyy-MM-dd}");


### Try it


In [ ]:
string query = "show emails for rick, summer and morty hired before 2024 in engineering";
var qspec = BuildQuerySpec(query);
Console.WriteLine(qspec);
var filtered = ExecuteQuery(qspec, employees).ToList();
foreach (var e in filtered) Console.WriteLine($"{e.DisplayName} — {e.Email} — {e.Department} — {e.Role} — {e.OriginalHireDate:yyyy-MM-dd}");
